__Vi bygger om logiken för att plocka ut 14 hästar när vi plockat bort de hästar som är i VNUM (1,2,3).__

In [1]:
# Common imports
import numpy as np
import os
import pandas as pd


In [2]:
# Necessary Sklearn objects used in the analysis
from sklearn.metrics import roc_curve, auc
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn import preprocessing

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
# DataPiplene är en pythonfil som innehåller den egenkostruerade klassen DataFrameSelector
from DataPipeline import DataFrameSelector
from sklearn.externals import joblib

__Läser in filen som ska köras__

In [4]:
df1 = pd.read_excel('RealDataV75TillUffeBO20190615.xlsx')

In [5]:
# Plockar bort hästara i VNUM (1,2,3) som inte ska vara med
df1 = df1[~df1.VNUM.isin([1,2,3])]    

In [6]:
df1 = df1.sort_values(['Lopp','Hast'], ascending = ['True','False']).reset_index()

In [7]:
# Importera model som ska användas
rf = joblib.load('Travmodel_v7.pkl')


In [8]:
# Import av datapipeline som ska användas
full_pipeline = joblib.load('Pipeline_v7.pkl')

In [19]:
df1.rename(columns = {'v75PROC':'V75PROC'}, inplace = True)

__Kör modellen och plockar fram de 14 hästarna med högst score__

In [20]:
df1['GRUPP'] = df1.GRUPP.astype('object')
features = full_pipeline.fit_transform(df1)

In [21]:
predict = rf.predict_proba(features)

predict_cutoff = pd.DataFrame({'Prob0':predict[:,0],'Prob1':predict[:,1]}).sort_values('Prob1', ascending = False).iloc[14]['Prob1']

y_pred = np.where( predict[:,1] > predict_cutoff,1,0)

In [22]:
# Gör om scorad array till en lista - multidimensionell
stack = predict.tolist()

In [23]:
# Sedan plockar vi den scorade sannolikheten att vara vinnare - 1
# Transformerar denna sannolikhet till en dataframe
last = []
for x in stack:
    last.append(x[1])
scored = {'Score':last}
# Konverterar till Dataframe
df_scored = pd.DataFrame.from_dict(scored)

In [24]:
# Gör om till lista och stoppar in i ett dictionary
pred = y_pred.tolist()
prediction = {'Predict':pred}
# Konverterar till Dataframe
df_pred = pd.DataFrame.from_dict(prediction)

In [25]:
df_predict1 = df1.merge(df_pred, right_index = True, left_index = True)
df_predict2 = df_predict1.merge(df_scored,  right_index = True, left_index = True) 

In [26]:
df1_out = df_predict2[df_predict2.Predict == 1][['Lopp','Hast','Score']]. \
sort_values(['Lopp','Score'], ascending = [True,False])
df1_out

,Lopp,Hast,Score
0,1,2,0.053398
5,1,9,0.042427
9,2,1,0.100630
20,2,13,0.093623
11,2,3,0.048240
15,2,7,0.048082
27,3,9,0.049223
32,4,3,0.129746
31,4,2,0.066411
37,4,11,0.063925


In [37]:
len(df1_out)

14

__Export till excel__

In [38]:
df1_out.to_excel('Outtake201906015.xlsx', index = False)